In [29]:
#HW functions to use in midterm

import numpy as np
import math as m

#initialize k_local
def define_ke_local(E, A, L):
    k = np.zeros((4,4))
    
    k[0][0] = (E*A/L)
    k[2][2] = (E*A/L)
    k[0][2] = (-E*A/L)
    k[2][0] = (-E*A/L)
    
    return k

def define_rotation(theta):
    unit_rot = [[np.cos(theta), np.sin(theta)], [-np.sin(theta), np.cos(theta)]]
    beta = np.zeros((4,4))
    beta[0:2,0:2] = unit_rot
    beta[2:4,2:4] = unit_rot
    return beta

def calc_length(i_coords, j_coords):
    return np.linalg.norm((j_coords - i_coords))

def calc_orientation(i_coords, j_coords):
    delta = j_coords - i_coords
    return np.arctan2(delta[1], delta[0])

def calc_dof(i_node_num, j_node_num):
    dof_i = [i_node_num * 2, i_node_num * 2 + 1]
    dof_j = [j_node_num * 2, j_node_num * 2 + 1]
    dof = dof_i + dof_j
    return dof

def define_ke_global(ke_local, beta):
    ke_global = np.dot((np.dot(np.transpose(beta), ke_local)), beta)
    return ke_global


def assemble_big_mac(num_dof, ke_global, global_dof):
    K = np.zeros((num_dof, num_dof))
    for i in range(len(ke_global)):
        K[global_dof[i][0]:(global_dof[i][1]+1),global_dof[i][0]:(global_dof[i][1]+1)] += ke_global[i][0:2,0:2]
        K[global_dof[i][0]:(global_dof[i][1]+1),global_dof[i][2]:(global_dof[i][3]+1)] += ke_global[i][0:2,2:4]
        K[global_dof[i][2]:(global_dof[i][3]+1),global_dof[i][0]:(global_dof[i][1]+1)] += ke_global[i][2:4,0:2]
        K[global_dof[i][2]:(global_dof[i][3]+1),global_dof[i][2]:(global_dof[i][3]+1)] += ke_global[i][2:4,2:4]
    return K

In [39]:
import csv
nodes = np.array([[0,0], [60,0], [108,0], [156,0], [204,0], [252,0], [330,0],[348,0],[396,0],
                 [444,0],[492,0],[564,0],[30,-34.292],[78,-34.292],[126,-34.292],[174,-34.292],
                 [222,-34.292],[270,-34.292],[318,-34.292],[366,-34.292],[414,-34.292],
                 [462,-34.292],[510,-34.292]])
eldata = np.genfromtxt('EN.560.csv', delimiter=',', skip_header=1, usecols=(1,2,3,4))
for i in range(len(eldata)):
    eldata[i][0] -= 1
    eldata[i][1] -= 1
    eldata[i][2] -= 1
    eldata[i][3] -= 1
section_props = [0.877, 0.3]
mat_props = [29500000.0]
A = [section_props[int(eldata[i][3])] for i in range(len(eldata))]
E = [mat_props[int(eldata[i][2])] for i in range(len(eldata))]
L = [calc_length(nodes[int(eldata[i][0])], nodes[int(eldata[i][1])]) for i in range(len(eldata))]
theta = [calc_orientation(nodes[int(eldata[i][0])], nodes[int(eldata[i][1])]) for i in range(len(eldata))]
ke_local = [define_ke_local(E[i], A[i], L[i]) for i in range(len(L))]
beta = [define_rotation(theta[i]) for i in range(len(theta))]
ke_global = [define_ke_global(ke_local[i], beta[i]) for i in range(len(ke_local))]
element_global_dof = [calc_dof(int(eldata[i][0]), int(eldata[i][1])) for i in range(len(eldata))]

num_dof = np.shape(nodes)[0] * 2
K = assemble_big_mac(num_dof, ke_global, element_global_dof)

s = [0,1,23]
u = np.zeros(np.size(nodes))
#assume windload of 300lbs/ft
w = 300.0 / 12
F = np.zeros(np.size(nodes))
for i in range(1,12):
    F[(2* i) - 1] += w * L[i-1]/2
    F[(2*i) + 1] += w * L[i-1]/2

temp_p = []
for i in range(num_dof):
    temp_p.append(i)
p = np.setdiff1d(temp_p, s)

pindex = np.ix_(p,p)
sindex = np.ix_(s,s)
psindex = np.ix_(p,s)
spindex = np.ix_(s,p)
Kpp = K[pindex]
Kss = K[sindex]
Kps = K[psindex]
Ksp = K[spindex]

Fp = F[p]
us = u[s]
up = np.dot(np.linalg.inv(Kpp), (Fp - np.dot(Kps, us)))
u[p] = up

Fs = np.dot(Ksp, up) + np.dot(Kss, us)
u_element_global = [u[element_global_dof[i]] for i in range(len(element_global_dof))]
displacement = [np.dot(beta[i], u_element_global[i]) for i in range(len(u_element_global))]
P = [np.dot(ke_local[i], displacement[i]) for i in range(len(ke_local))]

print("Max deflection:\n", max(u), "in.\n")
print("Force at element 9:\n", P[8][2], "lbs.")
print("Local deflection matrix:\n", displacement, "\n")
print("Global deflection matrix:\n", u_element_global, "\n")
print("Internal Forces:\n", P, "\n")
print("Big Mac:\n", K, "\n")

Max deflection:
 2.4857886518655197 in.

Force at element 9:
 22439.636066722276 lbs.
Local deflection matrix:
 [array([0.        , 0.        , 0.01278199, 0.84151842]), array([0.01278199, 0.84151842, 0.03805382, 1.43605846]), array([0.03805382, 1.43605846, 0.07501205, 1.920776  ]), array([0.07501205, 1.920776  , 0.1205403 , 2.2684078 ]), array([0.1205403 , 2.2684078 , 0.17152219, 2.46041483]), array([0.17152219, 2.46041483, 0.25757241, 2.41685842]), array([0.25757241, 2.41685842, 0.27750319, 2.33395408]), array([0.27750319, 2.33395408, 0.32614781, 2.03679912]), array([0.32614781, 2.03679912, 0.36778059, 1.59710576]), array([0.36778059, 1.59710576, 0.39928517, 1.03995619]), array([0.39928517, 1.03995619, 0.42623691, 0.        ]), array([0.42189741, 0.42635031, 0.40144622, 1.07316017]), array([0.40144622, 1.07316017, 0.36813999, 1.63025408]), array([0.36813999, 1.63025408, 0.3250951 , 2.0662794 ]), array([0.3250951 , 2.0662794 , 0.27542792, 2.35724447]), array([0.27542792, 2.35724447, 0